### 1.1 Hbase基本概念
#### 一. 为什么要用hbase
1. 相比于HDFS  
 * HDFS适合批处理场景, 但不支持数据的随机查找  
 * HDFS不支持数据的更新  
2. 相比于Hive  
 * Hive适合批处理的数据分析场景 
 * Hive不适合实时的数据访问
3. Hbase的特点  
 顺序写, 随机度, 可擦写, 可删除

#### 二. Hbase概念模型特点  
1. Hbase vs Mysql  
 1. 列  
    * Hbase的列可以无限扩充  
    * Hbase的列没有类型  
    * Hbase的多个列构成一个列簇(column family)  
 2. 主键 vs 索引  
    * Hbase仅有一个主键  
    * Hbase存储的每条记录根据主键进行排序  
 3. 值  
    * Hbase的值是多版本的, 每个版本的值称为Cell  
    * Cell的数目(每个值的版本数)也没有限制    
2. Hbase魔性特点  
 1. 可扩展性强  
    * 支持数十亿行, 数百万列  
    * 支持数十万个版本  
 2. 可以存储很稀疏的数据  
    * 即使有90%的列是空值, Hbase也不会因此浪费存储空间  
 3. 支持随机读  
    * 根据主键会哦去一行数据  
 4. 支持scan  
    * 快速获取某些行区间内的数据  
    * 快速获取某几列的数据
3. 行健举例  
历史账单查询:  userid+time+id  
网络爬虫: URL+爬去实践+优先级  

#### 三. Hbase物理模型  
1. 行式存储 vs 列式存储  
 1. 行式数据库  
    * 没建索引的行式数据库, 查询会产生大量IO  
    * 建立索引和物理视图会话费大量时间和资源  
    * 若要提升查询性能, 数据库必须被大量膨胀  
 2. 列式数据库  
    * 数据按列单独存放  
    * 数据即是索引  
    * 若只查询指定的几列, 可以大量降低IO  
    * 每一列可由一个thread单独处理-查询的并发  
    * 因为相同列的数据在一起存储, 数据类型一致, 特征相似, 方便高效压缩存储  
2. Hbase的列簇式存储  
    * Hbase没有选择纯粹的列式存储, 因为Hbase支持上百万个列, 没格列单独形成文件的话会造成每个标的metadata信息多大, 而且会导致产生数百万个小文件, 降低查询效率
    * 采用列簇式存储: 同一个列簇的所有列的数据存放在一起; 不同列簇的数据在物理上分开  
3. 列簇是排好序的Cell构成的  
  1. Cell的构成  
    * row key  
    * column family  
    * qualifier(列)  
    * timestamp  
    * value  
   (其中,前4个构成key)  
  2. Cell排序规则  
    * row key: 升序  
    * column family: 升序  
    * qualifier: 升序  
    * timestamp: 姜旭  
  3. Cell支持二分查找


#### 四. Hbase划分子表  
1. Region模型  
    * Hbase把一张大表拆分成多个子表, 每个子表成为一个Region  
    * 不同Region放在不同机器上  
    * 子表按照行健范围划分.   
     不存在一个row key的数据被划分到2个不同region的情况  
    * 每个Region都包含所有列表
2. Region的存储思路  
    * Region内部包含多个列簇, 每个列簇被一个Store存储  
    * 每个Store把数据分为两部分: MemStore和StoreFile  
    * MemStore只存储新增的和修改过的数据, 并在内存满的时候将数据刷新到StoreFile
    

#### 五. Hbase的特点

1. Hbase应用场景  
  1. 用户画像
     * 用户画像中用户的特征有几十万个, 但大部分特征都是空的, 是一个稀疏的矩阵
     * mysql存储上亿行, 几十万列的情况会造成巨大浪费
  2. 网页搜索倒排索引
  3. 淘宝交易数据
2. hbase是强一致性的
3. 如果需要更新数据, 或进行随机读写, Hbase比hdfs更合适
4. Hbase的字段部分数据类型, 全都按照字节存储, 需要上次应用系统按照类型翻译字节
5. Hbase只有聚簇索引,没有二级索引: 
    * 聚簇索引: 把这个字段抽取出来, 在这个字段上遍历查找, 而不用去全表扫描, 减小磁盘IO
    * 二级索引: 把索引字段排序后进行索引
6. null记录, 在Hbase中不占空间
7. Hbase不是列式存储,而是列族式存储
    * 把经常需要一起读写的业务字段设置成一个列族